# IMPORT

In [8]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec
import scipy
from scipy.signal import argrelextrema
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import numpy as np
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from __future__ import division
from astropy.stats import RipleysKEstimator
from sklearn.cluster import DBSCAN
from __future__ import division
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# Spatial Analysis 

In [14]:
def resize(shape,size):
    while shape/size!=shape//size:
        shape=shape+1
    return(shape)

In [15]:
def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""

    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))

In [16]:
def slice_images(paths_data,paths_mask,size=50):
    out=[]
    for el1,el2 in zip(paths_data,paths_mask):
        mtx=np.load(el1)
        mask=np.load(el2)
        zeros=np.zeros((resize(mtx.shape[0],size=size),resize(mtx.shape[0],size=size)))
        zeros[0:mtx.shape[0],0:mtx.shape[0]]=mtx[0:mtx.shape[0],0:mtx.shape[0]]
        
        mask_zeros=np.full((resize(mtx.shape[0],size=size),resize(mtx.shape[0],size=size)), False, dtype=bool)
        mask_zeros[0:mtx.shape[0],0:mtx.shape[0]]=mask[0:mask.shape[0],0:mask.shape[0]]
        
        a=split(zeros,size,size)
        b=split(mask_zeros,size,size)
        for slice1,slice2 in zip(a,b):
            if len(np.where(slice2==True)[0])==size*size:
                out.append(slice1)
    return(out)

In [17]:
def L_function(crops_array,treshold,area,radii):
    out=[]
    out_df=pd.DataFrame()
    Kest = RipleysKEstimator(area=area)
    for im in crops_array:
        z = np.random.uniform(low=0, high=area, size=(len(np.where(im>treshold)[0]), 2))
        A=np.where(im>treshold)
        data=np.concatenate((np.resize(A[0], len(A[0])).reshape(len(A[0]),1),np.resize(A[1], len(A[1])).reshape(len(A[1]),1)),axis=1)
        L=Kest.Lfunction(data,radii)
        L_rand=Kest.Lfunction(z,radii)
        Norm=L/L_rand
        out.append(Norm)
        tmp_df=pd.DataFrame(Norm)
        tmp_df[1]=np.array([x for x in range(len(radii))])
        out_df=pd.concat([out_df,tmp_df])
    return(np.nanmean(out,axis=0),out_df)

In [18]:
def pairCorrelationFunction_2D(x, y, S, rMax, dr):
    """Compute the two-dimensional pair correlation function for a set of
    spherical particles contained in a sqare with side length S.  This simple
    function finds reference particles such that a sphere of radius rMax drawn
    around the particle will fit entirely within the cube, eliminating the need
    to compensate for edge effects.  If no such particles exist, an error is
    returned.  Try a smaller rMax...or write some code to handle edge effects! ;)
    Arguments:
        x               an array of x positions of centers of particles
        y               an array of y positions of centers of particles
        S               length of each side of the cube in space
        rMax            outer diameter of largest spherical shell
        dr              increment for increasing radius of spherical shell
    Returns a tuple: (g, radii, interior_indices)
        g(r)            a numpy array containing the correlation function g(r)
        radii           a numpy array containing the radii of the
                        spherical shells used to compute g(r)
        reference_indices   indices of reference particles
    """
    from numpy import zeros, sqrt, where, pi, mean, arange, histogram

    # Find particles which are close enough to the cube center that a sphere of radius
    # rMax will not cross any face of the cube
    bools1 = x > rMax
    bools2 = x < (S - rMax) 
    bools3 = y > rMax
    bools4 = y < (S - rMax)

    interior_indices, = where(bools1 * bools2 * bools3 * bools4)
    num_interior_particles = len(interior_indices)

    if num_interior_particles < 1:
        raise  RuntimeError ("No particles found for which a sphere of radius rMax\
                will lie entirely within a cube of side length S.  Decrease rMax\
                or increase the size of the cube.")

    edges = arange(0., rMax + 1.1 * dr, dr)
    num_increments = len(edges) - 1
    g = zeros([num_interior_particles, num_increments])
    radii = zeros(num_increments)
    numberDensity = len(x) / S**2

    # Compute pairwise correlation for each interior particle
    for p in range(num_interior_particles):
        index = interior_indices[p]
        d = sqrt((x[index] - x)**2 + (y[index] - y)**2 )
        d[index] = 2 * rMax

        (result, bins) = histogram(d, bins=edges, normed=False)
        g[p,:] = result / numberDensity

    # Average g(r) for all interior particles and compute radii
    g_average = zeros(num_increments)
    for i in range(num_increments):
        radii[i] = (edges[i] + edges[i+1]) / 2.
        rOuter = edges[i + 1]
        rInner = edges[i]
        g_average[i] = mean(g[:, i]) / (4.0 / 3.0 * pi * (rOuter**3 - rInner**3))

    return (g_average, radii, interior_indices)
    # Number of particles in shell/total number of particles/volume of shell/number density
    # shell volume = 4/3*pi(r_outer**3-r_inner**3)

In [19]:
def RDF_function(crops_array,treshold,area,rMax,dr):
    out=[]
    out_df=pd.DataFrame()
    for im in crops_array:
        z = np.random.uniform(low=0, high=area, size=(len(np.where(im>treshold)[0]), 2))
        A=np.where(im>treshold)
        data=np.concatenate((np.resize(A[0], len(A[0])).reshape(len(A[0]),1),np.resize(A[1], len(A[1])).reshape(len(A[1]),1)),axis=1)
        RDF=pairCorrelationFunction_2D(data[:,0],data[:,1],S=area,rMax=rMax,dr=dr)
        RDF_rand=pairCorrelationFunction_2D(z[:,0],z[:,1],S=area,rMax=rMax,dr=dr)
        Norm=RDF[0]/RDF_rand[0]
        out.append(Norm)
        tmp_df=pd.DataFrame(Norm,)
        tmp_df[1]=np.array([x for x in range(rMax+1)])
        out_df=pd.concat([out_df,tmp_df])
    return(np.nanmean(out,axis=0),out_df)

# Nanodomain Analysis

In [45]:
def k_mean_distance(data, cx, cy, i_centroid):
    """data - table after clustering
    cx and cy coordinates of i centroid
    i_centroid - label of centroid
    in sklearn.Kmeans cluster_centers_ ordered by it's label e.g (0,1,2...N)"""
    # Calculate Euclidean distance for each data point assigned to centroid
    distances=[np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[data[4]==i_centroid][[1,2]].values]
    # return all values
    return distances

def k_mean_distance_SD(data, cx, cy, i_centroid):
    """data - table after clustering
    cx and cy coordinates of i centroid
    i_centroid - label of centroid
    in sklearn.Kmeans cluster_centers_ ordered by it's label e.g (0,1,2...N)"""
    # Calculate Euclidean distance for each data point assigned to centroid
    distances=[np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[data[4]==i_centroid][[1,2]].values]
    # return SD to the center
    return np.std(distances)

In [46]:
def K_means_nanodomains(mtx,treshold,size):
    """find nanodomains in nuclei
    return table with x,y,Intensity and cluster label and cluster_centers, 
    the last sorted by cluster num (0,1,2..)
    return table with lables and cluster centers"""
    #drop all intensities that < treshold
    locs=np.where(mtx>treshold)
    zeros=np.zeros(mtx.shape)
    for x,y in zip(locs[0],locs[1]):
        zeros[x,y]=mtx[x,y]
    mtx=zeros
    
    i=[]
    for x,y in zip(locs[0],locs[1]):
        i.append(mtx[x,y])
    I=np.array(i)
    
    #generate table of particles coordinates
    table=pd.DataFrame()
    table[1]=locs[0]
    table[2]=locs[1]
    table[3]=I
    # find local maximas
    detected_peaks=peak_local_max(mtx,size)
    
    #make a data table
    data=pd.DataFrame(detected_peaks,columns=['y','x'])
    #merge adjacent
    # Compute DBSCAN
    db = DBSCAN(eps=1, min_samples=1).fit(data)
    labels = db.labels_
    data['labels']=labels
    #get new peaks
    new_peaks=pd.DataFrame()
    for l in data['labels'].drop_duplicates():
        tmp=data[data['labels']==l]
        new_peaks=pd.concat([new_peaks,pd.DataFrame(np.array([tmp.mean(axis=0).values]))])
    new_peaks.columns=data.columns
    kmeans = KMeans(n_clusters=len(new_peaks),init=new_peaks[['x','y']], \
                    random_state=0,precompute_distances='auto',algorithm='elkan').fit(table[[1,2]])
    table[4]=kmeans.labels_
    table.sort_values(by=4,inplace=True)
    cluster_centers=kmeans.cluster_centers_
    
    return(table,cluster_centers)

def K_means_nanodomains_SD(mtx,treshold,size):
    """find nanodomains in nuclei
    return SD of distances of cluster points to cluster center"""
    #drop all intensities that < treshold
    locs=np.where(mtx>treshold)
    zeros=np.zeros(mtx.shape)
    for x,y in zip(locs[0],locs[1]):
        zeros[x,y]=mtx[x,y]
    mtx=zeros
    
    i=[]
    for x,y in zip(locs[0],locs[1]):
        i.append(mtx[x,y])
    I=np.array(i)
    
    #generate table of particles coordinates
    table=pd.DataFrame()
    table[1]=locs[0]
    table[2]=locs[1]
    table[3]=I
    # find local maximas
    detected_peaks=peak_local_max(mtx,size)
    
    #make a data table
    data=pd.DataFrame(detected_peaks,columns=['y','x'])
    #merge adjacent
    # Compute DBSCAN
    db = DBSCAN(eps=1, min_samples=1).fit(data)
    labels = db.labels_
    data['labels']=labels
    #get new peaks
    new_peaks=pd.DataFrame()
    for l in data['labels'].drop_duplicates():
        tmp=data[data['labels']==l]
        new_peaks=pd.concat([new_peaks,pd.DataFrame(np.array([tmp.mean(axis=0).values]))])
    new_peaks.columns=data.columns
    kmeans = KMeans(n_clusters=len(new_peaks),init=new_peaks[['x','y']], \
                    random_state=0,precompute_distances='auto',algorithm='elkan').fit(table[[1,2]])
    table[4]=kmeans.labels_
    table.sort_values(by=4,inplace=True)
    cluster_centers=kmeans.cluster_centers_
    
    #SD for all clusters
    SD=[]
    for i, (cx, cy) in enumerate(cluster_centers):
        SD.append(k_mean_distance_SD(table,cx,cy,i))
    
    return(np.array(SD))

def k_mean_cluster_1NN(data, cx, cy):
    """data - table of cluster centers
    cx and cy coordinates of i center
    in sklearn.Kmeans cluster_centers_ ordered by it's label e.g (0,1,2...N)"""
    # Calculate Euclidean distance for each data point assigned to centroid
    distances=[np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data]
    # return all values
    distances=np.sort(distances)
    #first element is zero cluster with itself,second is minimum
    return distances[1]

# Additional functions with filtering option (by nanodomain size) for nanodomain analysis

In [51]:
def filter_by_cluster_size(data,data_cluster_centers,min_size):
    out=pd.DataFrame()
    out_clusters=[]
    j=0
    for i, (cx, cy) in enumerate(data_cluster_centers):
        tmp=data[data[4]==i]
        if len(tmp)>=min_size:
            j=j+1
            tmp.loc[:,4]=np.array([j]*len(tmp))
            tmp.loc[:,4]=tmp[4]-1
            out=pd.concat([out,tmp])
            out_clusters.append([cx,cy])
    return(out,np.array(out_clusters))

def k_mean_distance(data, cx, cy, i_centroid):
    """data - table after clustering
    cx and cy coordinates of i centroid
    i_centroid - label of centroid
    in sklearn.Kmeans cluster_centers_ ordered by it's label e.g (0,1,2...N)"""
    # Calculate Euclidean distance for each data point assigned to centroid
    distances=[np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[data[4]==i_centroid][[1,2]].values]
    # return all values
    return distances

def k_mean_distance_SD(data, cx, cy, i_centroid):
    """data - table after clustering
    cx and cy coordinates of i centroid
    i_centroid - label of centroid
    in sklearn.Kmeans cluster_centers_ ordered by it's label e.g (0,1,2...N)"""
    # Calculate Euclidean distance for each data point assigned to centroid
    distances=[np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[data[4]==i_centroid][[1,2]].values]
    # return SD to the center
    return np.std(distances)

def K_means_nanodomains_filter(mtx,treshold,size,min_size):
    """find nanodomains in nuclei
    return table with x,y,Intensity and cluster label and cluster_centers, 
    the last sorted by cluster num (0,1,2..)
    return table with lables and cluster centers"""
    #drop all intensities that < treshold
    locs=np.where(mtx>treshold)
    zeros=np.zeros(mtx.shape)
    for x,y in zip(locs[0],locs[1]):
        zeros[x,y]=mtx[x,y]
    mtx=zeros
    
    i=[]
    for x,y in zip(locs[0],locs[1]):
        i.append(mtx[x,y])
    I=np.array(i)
    
    #generate table of particles coordinates
    table=pd.DataFrame()
    table[1]=locs[0]
    table[2]=locs[1]
    table[3]=I
    # find local maximas
    detected_peaks=peak_local_max(mtx,size)
    
    #make a data table
    data=pd.DataFrame(detected_peaks,columns=['y','x'])
    #merge adjacent
    # Compute DBSCAN
    db = DBSCAN(eps=1, min_samples=1).fit(data)
    labels = db.labels_
    data['labels']=labels
    #get new peaks
    new_peaks=pd.DataFrame()
    for l in data['labels'].drop_duplicates():
        tmp=data[data['labels']==l]
        new_peaks=pd.concat([new_peaks,pd.DataFrame(np.array([tmp.mean(axis=0).values]))])
    new_peaks.columns=data.columns
    kmeans = MiniBatchKMeans(n_clusters=len(new_peaks),init=new_peaks[['x','y']], \
                    random_state=0).fit(table[[1,2]])
    table[4]=kmeans.labels_
    table.sort_values(by=4,inplace=True)
    cluster_centers=kmeans.cluster_centers_
    #filter by cluster size
    return(filter_by_cluster_size(table,cluster_centers,min_size=min_size))

def K_means_nanodomains_SD_filter(mtx,treshold,size,min_size):
    """find nanodomains in nuclei
    return SD of distances of cluster points to cluster center"""
    #drop all intensities that < treshold
    locs=np.where(mtx>treshold)
    zeros=np.zeros(mtx.shape)
    for x,y in zip(locs[0],locs[1]):
        zeros[x,y]=mtx[x,y]
    mtx=zeros
    
    i=[]
    for x,y in zip(locs[0],locs[1]):
        i.append(mtx[x,y])
    I=np.array(i)
    
    #generate table of particles coordinates
    table=pd.DataFrame()
    table[1]=locs[0]
    table[2]=locs[1]
    table[3]=I
    # find local maximas
    detected_peaks=peak_local_max(mtx,size)
    
    #make a data table
    data=pd.DataFrame(detected_peaks,columns=['y','x'])
    #merge adjacent
    # Compute DBSCAN
    db = DBSCAN(eps=1, min_samples=1).fit(data)
    labels = db.labels_
    data['labels']=labels
    #get new peaks
    new_peaks=pd.DataFrame()
    for l in data['labels'].drop_duplicates():
        tmp=data[data['labels']==l]
        new_peaks=pd.concat([new_peaks,pd.DataFrame(np.array([tmp.mean(axis=0).values]))])
    new_peaks.columns=data.columns
    kmeans = MiniBatchKMeans(n_clusters=len(new_peaks),init=new_peaks[['x','y']], \
                    random_state=0).fit(table[[1,2]])
    table[4]=kmeans.labels_
    table.sort_values(by=4,inplace=True)
    cluster_centers=kmeans.cluster_centers_
    # filter by size
    ntable,ncluster_centers=filter_by_cluster_size(table,cluster_centers,min_size=min_size)
    #SD for all clusters
    SD=[]
    for i, (cx, cy) in enumerate(ncluster_centers):
        SD.append(k_mean_distance_SD(ntable,cx,cy,i))
    
    return(np.array(SD))

def k_mean_cluster_1NN(data, cx, cy):
    """data - table of cluster centers
    cx and cy coordinates of i center
    in sklearn.Kmeans cluster_centers_ ordered by it's label e.g (0,1,2...N)"""
    # Calculate Euclidean distance for each data point assigned to centroid
    distances=[np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data]
    # return all values
    distances=np.sort(distances)
    #first element is zero cluster with itself,second is minimum
    return distances[1]